In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = ('/Users/Beba/Documents/JupyterNotebooks/NY2013Crimes.csv')
raw_crimes = pd.read_csv(data, skiprows=4)
raw_crimes.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.0,NaN,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,"2,577",3,0.0,NaN,0,0,3,24,3,20,1,0.0
2,Akron Village,"2,846",3,0.0,NaN,0,0,3,16,1,15,0,0.0
3,Albany,"97,956",791,8.0,NaN,30,227,526,"4,090",705,"3,243",142,NaN
4,Albion Village,"6,388",23,0.0,NaN,3,4,16,223,53,165,5,NaN


In [4]:
raw_crimes.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'],
      dtype='object')

In [5]:
Property_Crime = pd.DataFrame()

In [6]:
Property_Crime['City'] = raw_crimes['City']
Property_Crime['Population'] = raw_crimes['Population']
Property_Crime['Robbery'] = raw_crimes['Robbery']
Property_Crime['Murder'] = raw_crimes['Murder and\nnonnegligent\nmanslaughter']

In [7]:
Property_Crime['Population'] = Property_Crime['Population'].str.replace(',','').astype(float)

In [8]:
Property_Crime['PopulationSQ'] = Property_Crime['Population'].apply(lambda x: x*x)

In [13]:
Property_Crime.head()

,City,Population,Robbery,Murder,PopulationSQ
0,Adams Village,1861.0,0,0.0,3.463321e+06
1,Addison Town and Village,2577.0,0,0.0,6.640929e+06
2,Akron Village,2846.0,0,0.0,8.099716e+06
3,Albany,97956.0,227,8.0,9.595378e+09
4,Albion Village,6388.0,4,0.0,4.080654e+07


In [58]:
Property_Crime['Robbery'] = Property_Crime['Robbery'].astype(str)

In [71]:
Property_Crime.head()

,City,Population,Robbery,Murder,PopulationSQ,Robbery_Category,Murder_Category
0,Adams Village,1861.0,0.0,0.0,3.463321e+06,0,0
1,Addison Town and Village,2577.0,0.0,0.0,6.640929e+06,0,0
2,Akron Village,2846.0,0.0,0.0,8.099716e+06,0,0
3,Albany,97956.0,227.0,8.0,9.595378e+09,1,1
4,Albion Village,6388.0,4.0,0.0,4.080654e+07,1,0


In [65]:
Property_Crime['Robbery_Category'] = np.where(Property_Crime['Robbery'].str.contains("0.0"), 0, 1)

In [68]:
Property_Crime['Murder'] = Property_Crime['Murder'].astype(str)
Property_Crime['Murder_Category'] = np.where(Property_Crime['Murder'].str.contains("0.0"), 0, 1)

In [70]:
Property_Crime.drop(['Robbery'], axis=1, inplace=True)
Property_Crime.drop([])

,City,Population,Murder,PopulationSQ,Robbery_Category,Murder_Category
0,Adams Village,1861.0,0.0,3.463321e+06,0,0
1,Addison Town and Village,2577.0,0.0,6.640929e+06,0,0
2,Akron Village,2846.0,0.0,8.099716e+06,0,0
3,Albany,97956.0,8.0,9.595378e+09,1,1
4,Albion Village,6388.0,0.0,4.080654e+07,1,0
5,Alfred Village,4089.0,0.0,1.671992e+07,1,0
6,Allegany Village,1781.0,0.0,3.171961e+06,0,0
7,Amherst Town,118296.0,1.0,1.399394e+10,1,1
8,Amityville Village,9519.0,0.0,9.061136e+07,1,0
9,Amsterdam,18182.0,0.0,3.305851e+08,1,0
